In [25]:
!pip install langchain


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install langchain_openai


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
import os
os.environ["OPENAI_API_KEY"] = "<paste-your-openai-key>"
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4.1")

In [55]:
from langchain.tools import tool

@tool
def create_calender_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str],
    location: str=""
) -> str:
    """Create a calender event. Requires exact ISO datetime format."""
    return f"Event created: {title} from  {start_time} to {end_time} with {len(attendees)} attendees"

@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    cc: list[str],
) -> str:
    """Send an email via email API. Requires properly fromatted addresses."""
    return f"Email sent to {', '.join(to)} - Subject: {subject}"

@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,
    duration_minutes: int
) -> list[str]:
    """Check calender availability for given attendees on a specific date."""
    return ["09:00", "14:00", "16:00"]
    

In [56]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

CALENDER_AGENT_PROMPT = (
    "You are a calender scheduling assistant. "
    "Parse natural language scheduling requests (e.g. , 'next Tuesday at 2pm')"
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calender_event to schedule events. "
    "Always confirm what was scheduled in your final response. "
)

calender_agent  = create_agent(
    model,
    tools=[create_calender_event, get_available_time_slots],
    system_prompt=CALENDER_AGENT_PROMPT,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"create_calender_event": True},
            description_prefix="Calender event pending approval",
        ),
    ],
)

In [57]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text."
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response. "
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"send_email": True},
            description_prefix="Outbound email pending approval",
        ),
    ],
)


In [58]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calender events using natural language.

    Use this when the user wants to create, modify, or check calender appointments.
    Handles date/time parsing, availability checking , and event creation.

    Input: Natural language scheduling request (e.g. , 'meeting with design team next
    Tuesday at 2pm')
    """

    result = calender_agent.invoke({
        "messages": [{"role":"user","content":request}]
    })

    return result["messages"][-1].text

@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user want to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about the meeting')
    """

    result = email_agent.invoke({
        "messages":[{"role":"user","content": request}]
    })
    return result["messages"][-1].text
    

In [59]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calender events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results."
    "When a request involves multiple actions, use multiple tools in sequence. "
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(),
)

    

In [62]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []
for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_FTJoK7ycwvC07bktNSaw2t1O)
 Call ID: call_FTJoK7ycwvC07bktNSaw2t1O
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour
  manage_email (call_lMWsnrlaSLSa3m86WhPGNUru)
 Call ID: call_lMWsnrlaSLSa3m86WhPGNUru
  Args:
    request: Send an email reminder to the design team about reviewing the new mockups before the meeting next Tuesday at 2pm

INTERRUPTED: 3103e57a984b71a24a355b53f8dbd252

INTERRUPTED: f1de53431f73e9d0244efcc61503fd3f


In [63]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

INTERRUPTED: 3103e57a984b71a24a355b53f8dbd252
Outbound email pending approval

Tool: send_email
Args: {'to': ['design-team@example.com'], 'subject': "Reminder: Review New Mockups Before Next Tuesday's Meeting", 'body': 'Dear Design Team,\n\nThis is a reminder to please review the new mockups ahead of our meeting next Tuesday at 2pm. Your feedback is important and will help us make final adjustments before the presentation.\n\nThank you for your attention to this matter.\n\nBest regards,\n[Your Name]', 'cc': []}

INTERRUPTED: f1de53431f73e9d0244efcc61503fd3f
Calender event pending approval

Tool: create_calender_event
Args: {'title': 'Meeting with the Design Team', 'start_time': '2024-06-18T14:00:00', 'end_time': '2024-06-18T15:00:00', 'attendees': ['design team']}



In [64]:
from langgraph.types import Command 

resume = {}
for interrupt_ in interrupts:
    if interrupt_.id == "2b56f299be313ad8bc689eff02973f16":
        # Edit email
        edited_action = interrupt_.value["action_requests"][0].copy()
        edited_action["args"]["subject"] = "Mockups reminder"
        resume[interrupt_.id] = {
            "decisions": [{"type": "edit", "edited_action": edited_action}]
        }
    else:
        resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

interrupts = []
for step in supervisor_agent.stream(
    Command(resume=resume), 
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================= Tool Message =================================
Name: schedule_event

The meeting with the design team has been scheduled for next Tuesday, June 18th, from 2:00 PM to 3:00 PM. Let me know if you need any additional details or changes!
================================= Tool Message =================================
Name: manage_email

Your reminder email to the design team has been sent with the following details:

- To: design-team@example.com
- Subject: Reminder: Review New Mockups Before Next Tuesday's Meeting
- Body: A reminder to review the new mockups before the meeting next Tuesday at 2pm, including a note that their feedback is important.

Let me know if you need to send any additional messages!
================================== Ai Message ==================================

The meeting with the design team has been scheduled for next Tuesday, June 18th, from 2:00 PM to 3:00 PM.

A reminder email has also been sent to the team, asking them to re